# Проект рекомендательной системы для ритейла

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install lightgbm

In [3]:
!pip install implicit==0.4.0

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3390403 sha256=29bc08f3d8df67c53728db3b75f3f808dfe59bda9a27c19223506f6f43963d80
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [4]:
# %load_ext autoreload
# %autoreload 2

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed = 42

# Модель второго уровня
import lightgbm as lgb

# Функции из 1-ого вебинара
import os, sys

# module_path = os.path.abspath(os.path.join(os.pardir))
module_path = os.path.abspath(os.path.join(f"{os.pardir}/content/drive/My Drive/Нейронные сети/Рекомендательные системы проект"))
if module_path not in sys.path:
    sys.path.append(module_path)


In [6]:
# %autoreload


from src.metrics import money_precision_at_k, money_recall_at_k, precision_at_k, recall_at_k
from src.utils import prefilter_items, postfilter_items
import src.recommenders as rcm

**Train-test datasets**

In [7]:
# data = pd.read_csv('../data/retail_train_sample.csv')

data = pd.read_csv(module_path+'/data/retail_train_sample.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

# подсчет цены 
data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))

max_svl = data['sales_value'].max()
min_svl = data['sales_value'].min()
delta = max_svl - min_svl

std_svl = data['sales_value'].std()
mean_svl = data['sales_value'].mean()
print('std_svl ', std_svl, mean_svl)


data['log_q'] = np.log(data['sales_value']+1)
data['norm_svalue'] = (data['sales_value'] - min_svl)*1.0/delta
data['std_svalue'] = (data['sales_value'] - mean_svl)*1.0/std_svl
# выборосим не нужные колонки
data = data[data.columns[1:]]

print(data.shape)


test_size_weeks = 6

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_val = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

print(data_train.shape)
data_train.head(2)

std_svl  4.3426020831287 3.0929014829982178
(227849, 16)
(207776, 16)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,log_q,norm_svalue,std_svalue
0,1078,35573861879,524,1082185,1,0.56,375,0.0,1440,76,0.0,0.0,0.56,0.444686,0.000886,-0.583268
1,324,29170411703,165,7168774,2,6.98,367,0.0,1115,24,0.0,0.0,3.49,2.076938,0.011048,0.895108


**Проверочный датасет**

In [8]:
# d_test = pd.read_csv('../data/retail_test1.csv')
d_test = pd.read_csv(module_path+'/data/retail_test1.csv')

d_test['price'] = d_test['sales_value'] / (np.maximum(d_test['quantity'], 1))
d_test.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0,6.29


**Продуктовый датасет**

In [9]:
# item_features = pd.read_csv('../data/product.csv')
item_features = pd.read_csv(module_path+'/raw_data/product.csv')

item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


**Датасет цен на items**

In [10]:
items_price = data.groupby('item_id')['price'].mean().reset_index()
items_price.columns = ['item_id', 'price']
items_price.head(2)

,item_id,price
0,26081,0.99
1,26540,0.97


In [11]:
# количество подкатегорий товаров
# item_features.commodity_desc.nunique()

In [12]:
def calc_model(data_train, data_test, items_price, item_features=None, take_n_popular=2600, **kwargs):
    
    print(kwargs)
    
    N = kwargs.get('N_rec', 5)
    print('filter')
    data = prefilter_items(data_train, take_n_popular, item_features)
    print('recommender')
    recommender = rcm.MainRecommender(data,
                                      kwargs.get('weighting', True), 
                                      kwargs.get('n_factors', 80), 
                                      kwargs.get('regularization', 0.001), 
                                      kwargs.get('iterations', 20), 
                                      num_threads=0 
                                     )

    # датасет для просчета метрик
    dd = data_test.groupby('user_id')
    result = dd['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    # добавим цены на реальные покупки
    
    print('result shape =', result.shape)
    
    # разбиваем на новых и старых пользователей
    new_users = np.array(list(set(result['user_id'].unique())-set(data['user_id'].unique())))
    r_new_usr = result.loc[result['user_id'].isin(new_users)]
    r_als = result.loc[~result['user_id'].isin(new_users)]
    
    # получаем рекомендации для новых пользователей
    top_n = recommender.overall_top_purchases[:N]
    sim_top_n = [recommender._get_similar_item(item) for item in top_n]
    r_new_usr['r_top'] = r_new_usr['user_id'].apply(lambda x: top_n)
    r_new_usr['recomendation'] = r_new_usr['r_top'] 
    r_new_usr['r_own'] = r_new_usr['user_id'].apply(lambda x: sim_top_n)

    # получаем рекомендации для старых пользователей
    r_als['recomendation'] = r_als['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N))
    r_als['r_own'] = r_als['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N))
    r_als['r_top'] = r_als['user_id'].apply(lambda x: recommender.overall_top_purchases[:N])
    
    result = r_als.append(r_new_usr)
    print('result shape =', result.shape)
    
    
    result['r_all'] = result['recomendation']+result['r_own']+result['r_top']
    result['r_all'] = result['r_all'].apply(lambda x: list(set(x)))
    print('r_all = ', len(result['r_all'].iloc[0]))
    
    
    # заполняем значение средних цен для рекомендаций
    def get_price(row):
        items = pd.Series(row['recomendation'], name='item_id')
        print(items)
        res = items_price.join(items,how='right', lsuffix='_l')
        return res
    
    result['recomendation_price'] = result['recomendation'].apply(lambda x: [items_price.loc[items_price['item_id']==i, 'price'].iloc[0] for i in x])
    p_at_k = result.apply(lambda row: precision_at_k(row['recomendation'], row['actual'], N), axis=1).mean()
    mp_at_k = result.apply(lambda row: money_precision_at_k(row['recomendation'], row['actual'], row['recomendation_price'], N), axis=1).mean()
    mp_at_o = result.apply(lambda row: money_precision_at_k(row['r_own'], row['actual'], row['recomendation_price'], N), axis=1).mean()
    mp_at_t = result.apply(lambda row: money_precision_at_k(row['r_top'], row['actual'], row['recomendation_price'], N), axis=1).mean()
    r_at_a = result.apply(lambda row: recall_at_k(row['r_all'], row['actual'], len(row['r_all'])), axis=1).mean()
    r_at_k = result.apply(lambda row: recall_at_k(row['recomendation'], row['actual'], N), axis=1).mean()
    
    
    print()
    print('***'*25)
    print('precision_at_k = ', p_at_k)
    print('money_precision_at_k = ', mp_at_k)
    print('money_precision_at_o = ', mp_at_o)
    print('money_precision_at_t = ', mp_at_t)
    
    print()
    print('recall_at_k = ', r_at_k)
    print('recall_at_a = ', r_at_a)
    print()
    
    return mp_at_k, result
#     return r_at_k


In [13]:
%%time
m_at_k_val, res_vaL = calc_model(data_train, data_val, items_price, item_features, take_n_popular=7650, weighting=True, n_factors=250, iterations = 126, N_rec = 200)

{'weighting': True, 'n_factors': 250, 'iterations': 126, 'N_rec': 200}
filter
recommender


GPU training requires factor size to be a multiple of 32 - 1. Increasing factors from 250 to 255.


start fit
LOG = 005
fit_own_recommender



result shape = (1979, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

result shape = (1979, 5)
r_all =  418

***************************************************************************
precision_at_k =  0.0060030318342597505
money_precision_at_k =  0.005456937308768207
money_precision_at_o =  0.006426664108470263
money_precision_at_t =  0.006471611243615553

recall_at_k =  0.1037381418871216
recall_at_a =  0.20317222810059424

CPU times: user 3min 45s, sys: 6.73 s, total: 3min 52s
Wall time: 3min 46s


In [14]:
res_val = res_vaL

In [15]:
res_val.columns

Index(['user_id', 'actual', 'recomendation', 'r_own', 'r_top', 'r_all',
       'recomendation_price'],
      dtype='object')

#### Подготовка 2 шага

In [16]:
# реальные покупки пользователей
result_lvl_1 = res_val.loc[:, ['user_id', 'actual']]
result_lvl_1.head(2)                               

,user_id,actual
0,1,"[1098248, 1017299, 1035805, 829563, 1077430, 9..."
1,2,"[980666, 1096261, 13876377, 901062, 13190188, ..."


In [17]:
# рекомендованные кандидаты
users_lvl_2 = res_val.loc[:, ['user_id', 'recomendation']]
users_lvl_2.rename(columns={'recomendation': 'candidates'}, inplace=True)
users_lvl_2.head(2)

,user_id,candidates
0,1,"[995242, 1081177, 1082185, 845078, 1004906, 99..."
1,2,"[951590, 899624, 940947, 909714, 990797, 95216..."


In [18]:
%%time

# разворачиваем кандидатов в один общий список
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1)
s= s.stack().reset_index(level=1, drop=True)
s.name = 'item_id'
print(s.shape)

# и джойним по индексу, предварительно дропнув исходных кандидатов
users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1  # фиктивная пересенная

users_lvl_2.shape

(395800,)
CPU times: user 1.03 s, sys: 33.7 ms, total: 1.07 s
Wall time: 1.06 s


In [19]:
# джойним кандидатов и реальные покупки и проставляем флаг таргета (угадали(1) или нет(0))

targets_lvl_2 = data_val[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [20]:
targets_lvl_2.head(2)

,user_id,item_id,target
0,1,995242,1.0
1,1,1081177,0.0


In [21]:
targets_lvl_2['user_id'].nunique()

1979

In [22]:
targets_lvl_2['target'].value_counts()

0.0    393424
1.0      2583
Name: target, dtype: int64

In [23]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [24]:
user_features = pd.read_csv(module_path+'/raw_data/hh_demographic.csv')
user_features.columns = [col.lower() for col in user_features.columns]
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [25]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,1,995242,1.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
1,1,1081177,0.0,2,PRODUCE,National,TOMATOES,TOMATOES VINE RIPE BULK,15 LB,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown


In [26]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2['target']

cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [27]:
# y_train = np.array(y_train).reshape(-1, 1).shape

In [28]:
train_data = lgb.Dataset(X_train, label=X_train.columns, categorical_feature=cat_feats)
train_data

In [29]:
%%time

model = lgb.LGBMClassifier(
    objective='binary', # lambdarank binary
#     max_depth=7, 
    n_estimators=750,
    categorical_column=cat_feats,
)


CPU times: user 52 µs, sys: 2 µs, total: 54 µs
Wall time: 59.4 µs


In [30]:
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


LGBMClassifier(boosting_type='gbdt',
               categorical_column=['manufacturer', 'department', 'brand',
                                   'commodity_desc', 'sub_commodity_desc',
                                   'curr_size_of_product', 'age_desc',
                                   'marital_status_code', 'income_desc',
                                   'homeowner_desc', 'hh_comp_desc',
                                   'household_size_desc', 'kid_category_desc'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=750,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [41]:
train_preds = model.predict(X_train, raw_score=True)
Pred_data = X_train[['user_id','item_id']].copy()
Pred_data['prediction'] = train_preds
True_pred = Pred_data[Pred_data['prediction']>0]

In [38]:
True_pred.head(3)

,user_id,item_id,prediction
2,1,1082185,0.881943
28,1,931136,0.762091
63,1,1098248,0.496265


In [40]:
True_pred.shape

(1347, 3)